In [8]:
import pandas as pd
import time
import sys
import requests
from bs4 import BeautifulSoup,Comment


def player_data(start_year,end_year,ps=False):
    if ps == False:
        season_type='leagues'
    else:
        season_type='playoffs'
    frames=[]
    for year in range(start_year,end_year+1):
        url = 'https://www.basketball-reference.com/'+season_type+'/NBA_'+str(year)+'_per_poss.html'
        
        df=pd.read_html(url)[0]
        df=df[df.Player!='Player']
        time.sleep(1)
        url2='https://www.basketball-reference.com/'+season_type+'/NBA_'+str(year)+'_advanced.html'
        df2=pd.read_html(url2)[0]
        df2=df2[df2.Player!='Player']

        

        year_df=df.merge(df2)
        unnamed=[col for col in year_df if 'unnamed' in col.lower()]
        year_df.drop(columns=unnamed,inplace=True)
        year_df['year']=year
        frames.append(year_df)
        print(year)

        time.sleep(2)
    return pd.concat(frames)



def pull_idinfo(start_year,end_year,ps=False):
    leagues = "playoffs" if ps else "leagues"
    frames = []
    for year in range(start_year, end_year+1):
        url = f"https://www.basketball-reference.com/{leagues}/NBA_{year}_totals.html"
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        response.encoding = 'utf-8' 
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find the specific table
        table = soup.find('table')
        
        # Get all rows from the table body
        rows = table.find('tbody').find_all('tr')
        
        # Iterate over each row and extract links and team acronyms
        data = []
        for row in rows:
            cells = row.find_all('td')
            if cells:
                player_cell = cells[0]  # Player name cell
                team_cell = cells[3]    # Team acronym cell (index 3 in the table)
                
                if player_cell.a:  # Check if the player cell has a link
                    player_name = player_cell.a.text
                    player_href = "https://www.basketball-reference.com" + player_cell.a['href']
                    team_acronym = team_cell.text if team_cell.text else "N/A"
                    
                    data.append([player_name, player_href, year, team_acronym])
        
        # Create DataFrame for the current year
        year_data = pd.DataFrame(data=data, columns=['player', 'url', 'year', 'team'])
        print(year)
        frames.append(year_data)
        time.sleep(2)
    
    return pd.concat(frames)


start_year=2025
end_year=2025

player_stats=player_data(start_year,end_year)

index_frame = pull_idinfo(start_year,end_year)
index_frame['bref_id']=index_frame['url'].str.split('/',expand=True)[5]
index_frame['bref_id']=index_frame['bref_id'].str.split('.',expand=True)[0]

index_frame.drop(columns=['team'],inplace=True)
index_frame.rename(columns={'Tm':'team','Player':'player'},inplace=True)
player_stats.rename(columns={'Tm':'team','Player':'player'},inplace=True)
player_stats=player_stats.merge(index_frame)
index_master=pd.read_csv('index_master.csv')
index_map=dict(zip(index_master['bref_id'],index_master['nba_id']))
player_stats['nba_id']=player_stats['bref_id'].map(index_map)
old_stats=pd.read_csv('player_stats.csv')
old_stats=old_stats[old_stats.year!=2025]
print(player_stats.columns)
print(old_stats.columns)
player_stats=pd.concat([old_stats,player_stats])

#player_stats.to_csv('player_stats_raw.csv',index=False)

#player_stats_ps=player_data(start_year,end_year,ps=True)
#player_stats_ps.to_csv('player_stats_ps.csv',index=False)
#player_stats_ps.to_csv('player_stats_ps_raw.csv',index=False)

player_stats.to_csv('player_stats.csv',index=False)
player_stats

2025
2025
Index(['Rk', 'player', 'Pos', 'Age', 'team', 'G', 'GS', 'MP', 'FG', 'FGA',
       'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'ORtg',
       'DRtg', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%',
       'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM',
       'DBPM', 'BPM', 'VORP', 'year', 'url', 'bref_id', 'nba_id'],
      dtype='object')
Index(['Rk', 'player', 'Pos', 'Age', 'team', 'G', 'GS', 'MP', 'FG', 'FGA',
       'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'ORtg',
       'DRtg', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%',
       'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM',
       'DBPM', 'BPM', 'VORP', 'year', 'url', 'bref_id', 'nba_id'],
      dtype='object')


,Rk,player,Pos,Age,team,G,GS,MP,FG,FGA,...,WS,WS/48,OBPM,DBPM,BPM,VORP,year,url,bref_id,nba_id
0,1,Zaid Abdul-Aziz,C,27,HOU,79,NaN,2459,6.2,13.5,...,6.5,0.126,0.9,0.1,1.0,1.8,1974,https://www.basketball-reference.com/players/a...,abdulza01,76002.0
1,2,Kareem Abdul-Jabbar,C,26,MIL,81,NaN,3548,12.2,22.6,...,18.4,0.25,5.0,2.5,7.5,8.6,1974,https://www.basketball-reference.com/players/a...,abdulka01,76003.0
2,3,Don Adams,SF,26,DET,74,NaN,2298,5.9,14.6,...,3.6,0.076,-2.2,0.7,-1.4,0.3,1974,https://www.basketball-reference.com/players/a...,adamsdo01,76012.0
3,4,Rick Adelman,PG,27,CHI,55,0.0,618,4.7,12.6,...,0.7,0.053,-3.5,1.8,-1.6,0.1,1974,https://www.basketball-reference.com/players/a...,adelmri01,76015.0
4,5,Lucius Allen,PG,26,MIL,72,NaN,2388,10.0,20.3,...,8.2,0.164,1.6,1.0,2.5,2.7,1974,https://www.basketball-reference.com/players/a...,allenlu01,76027.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428,430,James Wiseman,C,23,IND,1,0,5,19.4,38.8,...,0.0,.250,5.8,-5.7,0.2,0.0,2025,https://www.basketball-reference.com/players/w...,wisemja01,1630164.0
429,431,Delon Wright,PG,32,MIL,6,0,87,3.3,7.1,...,0.0,.007,-4.9,-1.0,-5.9,-0.1,2025,https://www.basketball-reference.com/players/w...,wrighde01,1626153.0
430,432,Guerschon Yabusele,PF,29,PHI,5,0,98,6.5,14.0,...,0.1,.065,-2.5,1.2,-1.3,0.0,2025,https://www.basketball-reference.com/players/y...,yabusgu01,1627824.0
431,433,Trae Young,PG,26,ATL,7,7,265,9.7,23.5,...,0.7,.121,4.9,-3.6,1.3,0.2,2025,https://www.basketball-reference.com/players/y...,youngtr01,1629027.0


In [ ]:
player_s

In [1]:
def league_average(league='leagues'):
    time.sleep(1)
    league_url='https://www.basketball-reference.com/'+league+'/NBA_stats_per_poss.html'
    frames=pd.read_html(league_url)
    league = frames[0]
    league.columns=league.columns.droplevel()
    league=league[league.Season!='Season']

    league2 = frames[1]
    league2.columns=league2.columns.droplevel()
    league2=league2[league2.Season!='Season']
    
    
    return league,league2
league,league2 = league_average()
league.to_csv('league_average.csv',index=False)

league2.to_csv('league_average_ps.csv',index=False)



NameError: name 'time' is not defined

In [ ]:
import unicodedata
import re
def transliterate(text):
    # Normalize to NFKD form (separate characters from diacritics)
    text = unicodedata.normalize('NFKD', text)
    # Remove diacritics
    text = ''.join([c for c in text if not unicodedata.combining(c)])
    # Replace remaining non-ASCII characters
    text = text.encode('ascii', 'ignore').decode('ascii')
    return text
def improved_transliterate(text):
    # Custom replacements
    custom_replacements = {
        'ö': 'o', 'Ö': 'O',
        'ä': 'a', 'Ä': 'A',
        'ü': 'u', 'Ü': 'U',
        'ß': 'ss',
        # Add more custom replacements as needed
    }
    
    for original, replacement in custom_replacements.items():
        text = text.replace(original, replacement)
    
    # Normalize to NFD form (separate characters from diacritics)
    text = unicodedata.normalize('NFD', text)
    # Remove diacritics
    text = ''.join([c for c in text if not unicodedata.combining(c)])
    # Replace remaining non-ASCII characters
    text = text.encode('ascii', 'ignore').decode('ascii')
    return text

In [ ]:
'''
print(len(players))
players.rename(columns={'Player':'player','Tm':'team'},inplace=True)
players['player']=players['player'].str.strip('*')
#players['player']=players['player'].str.encode('utf-8')

#players['player']=players['player'].apply(improved_transliterate)
player_index=pd.read_csv('index_master.csv')
#player_index['player']=player_index['player'].str.encode('utf-8')
#player_index['player']=player_index['player'].astype(str)
#player_index['player']=player_index['player'].apply(improved_transliterate)

players=players.merge(player_index,on=['player','team','year'])
print(len(players))
players.drop_duplicates(inplace=True)

players=players[(players.player!='Charles Jones')|(players.bref_id!='jonesch01')|(players.Pos!='PF')|(players.year!=1989)].reset_index(drop=True)

players=players[(players.player!='Charles Jones')|(players.bref_id!='jonesch02')|(players.Pos!='C')|(players.year!=1989)].reset_index(drop=True)
players
cloned=[]
for year in range(1974,2025):
    year_df=players[players.year==year].reset_index(drop=True)
    year_df['player_tm']=year_df['player']+year_df['team']
    vc=year_df['player_tm'].value_counts()
    
    vc=vc[vc>1]
    cloned.extend(vc.index)
    print(year)
    print(vc)
print(set(cloned))
'''

In [ ]:
'''players = pd.read_csv('player_stats_ps_raw.csv')
print(len(players))
players.rename(columns={'Player':'player','Tm':'team'},inplace=True)
players['player']=players['player'].str.strip('*')
#players['player']=players['player'].str.encode('utf-8')

#players['player']=players['player'].apply(improved_transliterate)
player_index=pd.read_csv('index_master.csv')
#player_index['player']=player_index['player'].str.encode('utf-8')
#player_index['player']=player_index['player'].astype(str)
#player_index['player']=player_index['player'].apply(improved_transliterate)

players=players.merge(player_index,on=['player','team','year'],how='left')
players.drop_duplicates(inplace=True)
print(len(players))
players=players[(players.player!='Charles Jones')|(players.bref_id!='jonesch01')|(players.Pos!='PF')|(players.year!=1989)].reset_index(drop=True)

players=players[(players.player!='Charles Jones')|(players.bref_id!='jonesch02')|(players.Pos!='C')|(players.year!=1989)].reset_index(drop=True)
players
#players.to_csv('player_stats_ps.csv',index=False)
missing_id=players[players.nba_id.isna()].reset_index(drop=True)
search=pd.read_csv('playersearch_index.csv')
missing_id.drop(columns=['url','bref_id','nba_id'],inplace=True)
temp_df=missing_id.merge(search,how='left').reset_index(drop=True)


luca=temp_df.loc[temp_df['player']=='Luca Vildoza'].reset_index(drop=True)
luca['url']=['https://www.basketball-reference.com/players/v/vildolu01.html']
luca['bref_id']=['vildolu01']
luca['nba_id']=[1630492.0]
players = pd.concat([players,temp_df,luca])
players=players.dropna(subset='nba_id')
players
players.to_csv('player_stats_ps.csv',index=False)
'''